In [2]:
import numpy as np
import os
import shutil
from pathlib import Path
from PIL import Image

from imageai.Detection.Custom import DetectionModelTrainer
from imageai.Detection.Custom import CustomObjectDetection

In [ ]:
root_annots_path = '/content/chess-piece-dectection/annotations/'
root_images_path = '/content/chess-piece-dectection/images/'

annots_path = sorted([i for i in Path(root_annots_path).glob('*.xml')])
images_path = sorted([i for i in Path(root_images_path).glob('*.png')])

n_imgs = len(images_path)
print('Количество картинок: ', n_imgs)

In [ ]:
classes = np.array(["white-pawn", "black-pawn",
                    "white-bishop", "black-bishop",
                    "white-knight", "black-knight",
                    "white-king", "black-king",
                    "white-queen", "black-queen",
                    "white-rook", "black-rook"])

In [ ]:
with open(annots_path[50], 'r') as f:
    print(f.read())

In [ ]:
# create foldes for train and test
os.makedirs('/content/chess-piece-dectection/train/images', exist_ok=True)
os.makedirs('/content/chess-piece-dectection/train/annotations', exist_ok=True)

os.makedirs('/content/chess-piece-dectection/validation/images', exist_ok=True)
os.makedirs('/content/chess-piece-dectection/validation/annotations', exist_ok=True)

os.makedirs('/content/chess-piece-dectection/test/images', exist_ok=True)
os.makedirs('/content/chess-piece-dectection/test/annotations', exist_ok=True)

In [ ]:
# делим картинки на трейн, валидацию и тест
n_split = n_imgs // 20


for i, (annot_path, img_path) in enumerate(zip(annots_path, images_path)):    
    if i > n_imgs:
        break
    # train-val-test split
    if i < n_split:
        shutil.copy(img_path, '/content/chess-piece-dectection/test/images/' + img_path.parts[-1])
        shutil.copy(annot_path, '/content/chess-piece-dectection/test/annotations/' + annot_path.parts[-1])
    elif n_split <= i < n_split*5:
        shutil.copy(img_path, '/content/chess-piece-dectection/validation/images/' + img_path.parts[-1])
        shutil.copy(annot_path, '/content/chess-piece-dectection/validation/annotations/' + annot_path.parts[-1])
    else:
        shutil.copy(img_path, '/content/chess-piece-dectection/train/images/' + img_path.parts[-1])
        shutil.copy(annot_path, '/content/chess-piece-dectection/train/annotations/' + annot_path.parts[-1])

In [ ]:
print('test: ', len(list(Path('/content/chess-piece-dectection/train/annotations/').glob('*.xml'))))
print('val: ', len(list(Path('/content/chess-piece-dectection/validation/annotations/').glob('*.xml'))))
print('test: ', len(list(Path('/content/chess-piece-dectection/test/annotations/').glob('*.xml'))))

In [ ]:
# скачиваем веса предобученной модели
%cd /content/chess-piece-dectection/
!wget wget https://pjreddie.com/media/files/yolov3.weights

In [ ]:
# скачиваем конвертер весов и конфиг https://github.com/qqwweee/keras-yolo3/find/master
!wget wget https://raw.githubusercontent.com/qqwweee/keras-yolo3/master/convert.py
!wget wget https://raw.githubusercontent.com/qqwweee/keras-yolo3/master/yolov3.cfg

In [ ]:
%run convert.py yolov3.cfg yolov3.weights yolov3.h5

In [ ]:
trainer = DetectionModelTrainer()
trainer.setModelTypeAsYOLOv3() # дообучать можно только 'большую' YOLO (разработчики)
trainer.setDataDirectory(data_directory='/content/chess-piece-dectection/') # где лежат даные
trainer.setTrainConfig(object_names_array=classes,
                       batch_size=8,
                       num_experiments=20,
                       # дообучаемся на их модели
                       train_from_pretrained_model='/content/chess-piece-dectection/yolov3.h5',
                       
                       # можно дообучаться со своей модели
                       #train_from_pretrained_model="imageai/data/models/detection_model-ex-009--loss-0024.110.h5")
                       )

In [ ]:
trainer.trainModel()

In [ ]:
# извлекаем путь к последней модели
model_path = sorted(list(Path('/content/chess-piece-dectection/models/').iterdir()))[-1]
model_path

In [ ]:
trainer = DetectionModelTrainer()
trainer.setModelTypeAsYOLOv3()
trainer.setDataDirectory(data_directory="/content/chess-piece-dectection/")
metrics = trainer.evaluateModel(model_path=model_path,
                                json_path="/content/chess-piece-dectection/json/detection_config.json",
                                iou_threshold=0.2,
                                object_threshold=0.3,
                                nms_threshold=0.5)

In [ ]:
input_image_path="/content/chess-piece-dectection/test/images/chess0.png"
output_image_path="/content/chess-piece-dectection/test/images/detected.jpg"

In [ ]:
detector = CustomObjectDetection()
detector.setModelTypeAsYOLOv3()
detector.setModelPath(model_path)
detector.setJsonPath("/content/chess-piece-dectection/json/detection_config.json")
detector.loadModel()
detections = detector.detectObjectsFromImage(minimum_percentage_probability=60,
                                             input_image=input_image_path,
                                             output_image_path=output_image_path)
for detection in detections:
    print(detection["name"], " : ", detection["percentage_probability"], " : ", detection["box_points"])

In [ ]:
Image.open(input_image_path)

In [ ]:
Image.open(output_image_path)

Вывод: обучение на 10 эпохах дало весьма посредственный результат. На 20 - значительно лучше. Это заметно и по отрисовке боксов и по тому как снижается лосс. Очевидно, при увеличении количества эпох качество будет повышаться.